# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from ipywidgets import IntProgress
from IPython.display import display

from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
import lightgbm as lgbm
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv('/datasets/autos.csv')
df.head(2)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Выполним предобработку данных, т.к. есть некорректные значения
- Удалим нулевые значения из столбца Price
- В столбце RegistrationYear есть выбросы. Минимальные и максимальные значения - 1000, 9999 выглядят неправдоподобными. Актуальные года регистриции автомобилей от 1973 до 2023. Остальные значения вне актуальных дат удалим, т.к. все автомобили до 1973 это уникальные случаи, по которым лучше не строить пронозы
- В столбце Power также присутствуют некорректные значение 0 и 20000 л.с. Корректными значениями будем считать интервал от 9 до 600 л.с. Строк с значениями до 9 л.с. 11%, заполним эти начения средними, чтобы не терять так много данных. Автомобили с мощностью больше 600 л.с. удалим, так как максимальная стоимость авто 20000 евро, что является недостаточным бюджетом для покупки сверхмощного авто. Значит в данных по мощности есть грубые ошибки
- В столбце RegistrationMounth есть нулевые значения. Удалим или заполним их в зависимости от общего числа
- В столбце NumberOfPictures все значения нулевые. Этот признак не будем использовать, как обучающий, так как кол-во фотографий не должно влиять на стоимость авто

- Обработаем пропуски
- Столбцы DataCrawled, PostalCode, LastSeen не информативные. Перед обучением модели уберем их из датасета или обучающих признаков

Удалим строки, где значение Price = 0

In [5]:
lendf=len(df['Price'])#переменная для удобства расчетов

In [6]:
(df.loc[df['Price']==0,'Price'].count()/lendf)*100# доля (в процентах) нулевых значений Price

3.0397692800442475

In [7]:
df=df.drop(df[df['Price']==0].index)

Удалим строки, где значения RegistrationYear 1973 < RegistrationYear < 2023

In [8]:
(df.loc[df['RegistrationYear']<1973,'RegistrationYear'].count()/len(df['RegistrationYear']))*100# доля (в процентах) RegisrtationYear < 1973

0.5320186148307464

In [9]:
(df.loc[df['RegistrationYear']>2023,'RegistrationYear'].count()/len(df['RegistrationYear']))*100# доля (в процентах) RegistrationYear > 2023

0.025029322141927896

In [10]:
df=df.drop(df[df['RegistrationYear']<1973].index)
df=df.drop(df[df['RegistrationYear']>2023].index)

In [11]:
df['DateCrawled'] = pd.to_datetime(df['DateCrawled'], format='%Y-%m-%d %H:%M:%S')
df['DateCrawled'].max()
df['DateCrawled'].min()

Timestamp('2016-03-05 14:06:22')

Заполним автомобили до 9 л.с. в столбце Power средним значением, - больше 1200 л.с. удалим

In [12]:
(df.loc[df['Power']<9,'Power'].count()/lendf)*100 #процент выбросов в столбце Power

10.112622718127149

In [13]:
(df.loc[df['Power']>600,'Power'].count()/lendf)*100#процент выбросов в столбце Power

0.09763833743922296

In [14]:
df=df.drop(df[df['Power']>600].index)

In [15]:
df.loc[df['Power']<9,'Power']=df['Power'].mean()

В столбце Registration Month 9% нулевых значений. Заполним их средними.

In [16]:
(df.loc[df['RegistrationMonth']==0,'RegistrationMonth'].count()/lendf)*100

9.12664482502702

In [17]:
df.loc[df['RegistrationMonth']==0,'RegistrationMonth']=df['RegistrationMonth'].mean()

Рассмотрим кол-во пропусков в исходных данных

In [18]:
round(df.isna().mean()*100)#процент пропусков в каждом столбце

DateCrawled           0.0
Price                 0.0
VehicleType          10.0
RegistrationYear      0.0
Gearbox               5.0
Power                 0.0
Model                 5.0
Kilometer             0.0
RegistrationMonth     0.0
FuelType              8.0
Brand                 0.0
Repaired             19.0
DateCreated           0.0
NumberOfPictures      0.0
PostalCode            0.0
LastSeen              0.0
dtype: float64

Пропуски в столбцах VehicleType, FuelType, GearBox, Model лучше удалить, так как это важные параметры для обучения, доля пропусков небольшая (до 10%) и восстановить их корректные значения не получится. Пропусков в Repaired много, заполним признаком 'unknown'

In [19]:
df.dropna(subset=['VehicleType','FuelType','Gearbox','Model'], inplace=True)

In [20]:
df['Repaired'].unique()

array([nan, 'no', 'yes'], dtype=object)

In [21]:
df['Repaired']=df['Repaired'].fillna('unknown')

Столбец DataCreated, перевели в тип DateTime

In [22]:
df['DateCreated'] = pd.to_datetime(df['DateCreated'], format='%Y-%m-%d %H:%M:%S')

Рассмотрим корреляционную матрицу

In [23]:
df.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.607163,0.504216,-0.391738,0.015708,NaN,0.066286
RegistrationYear,0.607163,1.000000,0.158943,-0.383266,0.001829,NaN,0.040038
Power,0.504216,0.158943,1.000000,0.103627,0.021546,NaN,0.052507
Kilometer,-0.391738,-0.383266,0.103627,1.000000,0.003827,NaN,-0.011861
RegistrationMonth,0.015708,0.001829,0.021546,0.003827,1.000000,NaN,-0.007689
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.066286,0.040038,0.052507,-0.011861,-0.007689,NaN,1.000000


Вывод
- Выполнили предобработку данных. Вcе типы данных корректные, кроме столбца DataCreated, его перевели в тип DateTime. Обработали пропуски и аномалии
- Определили неинформативные столбцы, которые не будем использовать для обучения модели. DateCrawled, RegistrationMonth,NumberOfPictures, PostalCode,LastSeen
- После предобработки сохранилось 78.4 % данных, 277944 строк, чего достаточно для качественного обучения модели
- Видим большую положительную корреляцию целевого признака со столбцами RegistrationYear, Power и сильную отрицательную корреляцию со столбцом Kilometer

## Обучение моделей

Подготовим датасет и выборки для обучения моделей. Датасет для модели назовем df1. Закодируем категориальные признаки.Отмасштабируем признаки для лучшего качества модели. Сделаем прогнозы и проверим метрику качества rmse

Удалим линшие признаки, которые не влиют на стоимость автомобиля

In [24]:
df1=df.drop(['DateCrawled','RegistrationMonth','NumberOfPictures','PostalCode','LastSeen','DateCreated'], axis=1)

In [25]:
df1=df1.reset_index(drop=True)
df1.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,Repaired
0,9800,suv,2004,auto,163.0,grand,125000,gasoline,jeep,unknown
1,1500,small,2001,manual,75.0,golf,150000,petrol,volkswagen,no
2,3600,small,2008,manual,69.0,fabia,90000,gasoline,skoda,no
3,650,sedan,1995,manual,102.0,3er,150000,petrol,bmw,yes
4,2200,convertible,2004,manual,109.0,2_reihe,150000,petrol,peugeot,no


Разделим признаки на обучающие и целевые

In [26]:
features=df1.drop('Price',axis=1)
target=df1['Price']

Закодируем категориальные признаки в обучающих признаках

In [27]:
columns=['VehicleType','Gearbox','Model','FuelType','Brand','Repaired']#категориальные признаки, которые нужно закодировать
encoder = OrdinalEncoder()
encoder.fit(features[columns])
features_ord=features.copy()
features_ord[columns]= pd.DataFrame(encoder.transform(features[columns]), columns = features[columns].columns)

In [28]:
features_train,features_valid,target_train, target_valid = train_test_split(features_ord,target, test_size=0.4, random_state=12345)

In [29]:
features_valid, features_test, target_valid, target_test=train_test_split(features_valid,target_valid, test_size=0.5, random_state=12345)

Модель LinearRegression

In [30]:
#%%time
start=time.time()
model_lr=LinearRegression()
model_lr.fit(features_train,target_train)
end=time.time()
time_lr_fit=end-start
print(f'{round(time_lr_fit,2)}')

0.04


In [31]:
#%%time
start=time.time()
predict_lr=model_lr.predict(features_valid)
end=time.time()
time_lr_predict=end-start
rmse_lr=mean_squared_error(target_valid,predict_lr)**0.5
print (f'rmse модели LinearRegression = {round(rmse_lr)}, \
время обучения модели = {round(time_lr_fit,2)} сек., \
время предсказания модели = {round(time_lr_predict,2)}сек.')

rmse модели LinearRegression = 2792, время обучения модели = 0.04 сек., время предсказания модели = 0.0сек.


<font color='steelblue'>Для финального теста использую выборку test.</font>

Выведем результат подбора наиболее эффективных гиперпараметров модели  RandomForestRegressor, ячейку с подбором переведу в формат NBConvert, чтобы не перегружать файл проекта вычислениями

Самые эффективные гиперпараметры для модели RFR: best_est - 105, best_depth - 19

Лучшее значние rmse на этапе  1551

Модель RandomForestRegressor

In [32]:
start=time.time()
model_rfr=RandomForestRegressor(n_estimators=105,max_depth=19,random_state=12345,n_jobs=-1)
model_rfr.fit(features_train,target_train)
end=time.time()
time_rfr_fit=end-start

In [33]:
start=time.time()
predict_rfr=model_rfr.predict(features_valid)
end=time.time()
time_rfr_predict=end-start
rmse_rfr=mean_squared_error(target_valid,predict_rfr)**0.5
print(f'rmse модели RandomForestRegressor = {round(rmse_rfr)}, время обучения модели = {round(time_rfr_fit,2)} сек., \
                                            время предсказания модели = {round(time_rfr_predict,2)}сек.')

rmse модели RandomForestRegressor = 1551, время обучения модели = 34.28 сек.,                                             время предсказания модели = 1.52сек.


Модель CatBoostRegressor

In [34]:
start=time.time()
model_cat=CatBoostRegressor(random_state=12345,learning_rate=0.3, verbose=100,n_estimators=1400)
model_cat.fit(features_train,target_train,eval_set=(features_valid,target_valid))
end=time.time()
time_cat_fit=end-start

0:	learn: 3721.5286127	test: 3721.7030368	best: 3721.7030368 (0)	total: 76.8ms	remaining: 1m 47s
100:	learn: 1611.7855436	test: 1638.2751913	best: 1638.2751913 (100)	total: 2.26s	remaining: 29.1s
200:	learn: 1531.5246012	test: 1582.5073251	best: 1582.5073251 (200)	total: 4.4s	remaining: 26.2s
300:	learn: 1487.4602500	test: 1556.7852547	best: 1556.7852547 (300)	total: 6.52s	remaining: 23.8s
400:	learn: 1457.3547963	test: 1541.2229988	best: 1541.0570046 (399)	total: 8.65s	remaining: 21.6s
500:	learn: 1430.9628171	test: 1531.2384068	best: 1531.2384068 (500)	total: 10.8s	remaining: 19.3s
600:	learn: 1410.5654832	test: 1523.8444655	best: 1523.7577039 (597)	total: 12.9s	remaining: 17.1s
700:	learn: 1393.6202923	test: 1517.1611449	best: 1516.9358566 (690)	total: 15s	remaining: 15s
800:	learn: 1378.9696027	test: 1513.0698907	best: 1513.0698907 (800)	total: 17.2s	remaining: 12.8s
900:	learn: 1364.8249725	test: 1509.7564770	best: 1509.6836384 (892)	total: 19.3s	remaining: 10.7s
1000:	learn: 1353

In [35]:
start=time.time()
predict_cat=model_cat.predict(features_valid)
end=time.time()
time_cat_predict=end-start
rmse_cat=mean_squared_error(target_valid,predict_cat)**0.5
print(f'rmse модели CatBoostRegressor = {round(rmse_cat)}, время обучения модели = {round(time_cat_fit,2)} сек., \
                                             время предсказания модели = {round(time_cat_predict,2)} сек.')

rmse модели CatBoostRegressor = 1502, время обучения модели = 31.77 сек.,                                              время предсказания модели = 0.09 сек.


Модель LightGBM

In [36]:
start = time.time()
model_light = lgbm.sklearn.LGBMRegressor(learning_rate=0.1, num_leaves=50)
model_light.fit(features_train, target_train,eval_set=(features_valid,target_valid))
end = time.time()
time_light_fit=end-start
time_light_fit

[1]	valid_0's l2: 1.83142e+07
[2]	valid_0's l2: 1.57692e+07
[3]	valid_0's l2: 1.36998e+07
[4]	valid_0's l2: 1.20113e+07
[5]	valid_0's l2: 1.06246e+07
[6]	valid_0's l2: 9.45905e+06
[7]	valid_0's l2: 8.50996e+06
[8]	valid_0's l2: 7.72166e+06
[9]	valid_0's l2: 7.062e+06
[10]	valid_0's l2: 6.50566e+06
[11]	valid_0's l2: 6.03381e+06
[12]	valid_0's l2: 5.64405e+06
[13]	valid_0's l2: 5.31072e+06
[14]	valid_0's l2: 5.03177e+06
[15]	valid_0's l2: 4.778e+06
[16]	valid_0's l2: 4.56672e+06
[17]	valid_0's l2: 4.38965e+06
[18]	valid_0's l2: 4.22171e+06
[19]	valid_0's l2: 4.08304e+06
[20]	valid_0's l2: 3.96041e+06
[21]	valid_0's l2: 3.85288e+06
[22]	valid_0's l2: 3.76093e+06
[23]	valid_0's l2: 3.67244e+06
[24]	valid_0's l2: 3.59444e+06
[25]	valid_0's l2: 3.53128e+06
[26]	valid_0's l2: 3.46435e+06
[27]	valid_0's l2: 3.40731e+06
[28]	valid_0's l2: 3.35316e+06
[29]	valid_0's l2: 3.29733e+06
[30]	valid_0's l2: 3.25468e+06
[31]	valid_0's l2: 3.21413e+06
[32]	valid_0's l2: 3.17568e+06
[33]	valid_0's l2: 3.

8.000999927520752

In [37]:
%%time
start = time.time()
predict_light= model_light.predict(features_valid)
end = time.time()
rmse_light=mean_squared_error(target_valid,predict_light)**0.5
time_light_predict=end-start
print(f'rmse модели LightGBM = {round(rmse_light)}, время обучения модели = {round(time_light_fit,2)} сек., \
                                            время предсказания модели = {round(time_light_predict,2)} сек.')

rmse модели LightGBM = 1596, время обучения модели = 8.0 сек.,                                             время предсказания модели = 0.5 сек.
CPU times: user 490 ms, sys: 0 ns, total: 490 ms
Wall time: 502 ms


## Анализ моделей

Сделаем сводную сравнительную таблицу по всем рассмотренным моделям. Будем сравнивать качество предсказания (RMSE), скорость обучения и прогноза

In [38]:
table_model=pd.DataFrame(index=['LinearRegression','RandomForestRegressor','CatBoostRegressor','LightGBM'],columns=['RMSE','время обучения', 'время предсказания'])

In [39]:
table_model['RMSE']['LinearRegression']=round(rmse_lr,2)
table_model['время обучения']['LinearRegression']=round(time_lr_fit,2)
table_model['время предсказания']['LinearRegression']=round(time_lr_predict,2)
table_model['RMSE']['RandomForestRegressor']=round(rmse_rfr,2)
table_model['время обучения']['RandomForestRegressor']=round(time_rfr_fit,2)
table_model['время предсказания']['RandomForestRegressor']=round(time_rfr_predict,2)
table_model['RMSE']['CatBoostRegressor']=round(rmse_cat,2)
table_model['время обучения']['CatBoostRegressor']=round(time_cat_fit,2)
table_model['время предсказания']['CatBoostRegressor']=round(time_cat_predict,2)
table_model['RMSE']['LightGBM']=round(rmse_light,2)
table_model['время обучения']['LightGBM']=round(time_light_fit,2)
table_model['время предсказания']['LightGBM']=round(time_light_predict,2)

In [40]:
table_model

,RMSE,время обучения,время предсказания
LinearRegression,2791.99,0.04,0.0
RandomForestRegressor,1550.95,34.28,1.52
CatBoostRegressor,1502.49,31.77,0.09
LightGBM,1595.81,8.0,0.5


Проведем тестирование нашей модели на тестовой выборке

In [41]:
start = time.time()
test_cat_predict=model_cat.predict(features_test)
end=time.time()
time_cat_test_predict=end-start
rmse_cat_test=mean_squared_error(target_test,test_cat_predict)**0.5
print(f'rmse модели CatBoostRegressor = {round(rmse_cat_test)}, \
                                             время предсказания модели = {round(time_cat_test_predict,2)} сек.')

rmse модели CatBoostRegressor = 1487,                                              время предсказания модели = 0.09 сек.


Вывод

- Видим, что по качеству предсказания лидером является модель CatBoostRegressor.
- Линейная регрессия является лидером по скорости, однако качество модели сильно уступает остальным, поэтому далее с ней не будем сравнивать
- CatBoostRegressor выглядит наиболее предопчтительной, так как является лидером и по скорости обучения, предсказания и по качеству предсказания, основываясь на метрику RMSE
- На финальном тестировании CatBoostRegressor получили хорошие показатели качества и скорости. rmse=1487, время предсказания модели 0.07 сек. Можно рекомендовать модель CatBoostRegressor к использованию